## DIA 025: Implementacion de Confirmacion de Email para Usuarios Registrados

1. Objetivos del Día 25
Configurar el Envío de Emails:

Integrar Flask-Mail para enviar correos electrónicos de confirmación a los usuarios registrados.
Actualizar el Modelo de Usuario:

Añadir campos para gestionar la confirmación de email.
Implementar el Flujo de Confirmación de Email:

Generar tokens seguros para la confirmación.
Crear endpoints para enviar y verificar tokens.
Actualizar el Frontend:

Permitir a los usuarios registrar y confirmar sus direcciones de email.
Actualizar las Pruebas Automatizadas:

Asegurar que el proceso de confirmación de email funciona correctamente.
2. Configurar el Envío de Emails con Flask-Mail
2.1. Instalar Dependencias Necesarias
Añade Flask-Mail a tu archivo requirements.txt:

plaintext
Copiar
Flask-Mail==0.9.1
itsdangerous==2.1.2
Instalar las Dependencias:

bash
Copiar
pip install -r requirements.txt
2.2. Configurar Flask-Mail en api.py
Actualiza tu archivo api.py para configurar Flask-Mail:

python
Copiar
# api.py

from flask_mail import Mail, Message
from itsdangerous import URLSafeTimedSerializer, SignatureExpired, BadSignature

# Configuración de Flask-Mail
app.config['MAIL_SERVER'] = os.getenv('MAIL_SERVER', 'smtp.gmail.com')
app.config['MAIL_PORT'] = int(os.getenv('MAIL_PORT', 587))
app.config['MAIL_USE_TLS'] = os.getenv('MAIL_USE_TLS', 'true').lower() in ['true', '1', 't']
app.config['MAIL_USERNAME'] = os.getenv('MAIL_USERNAME')  # Tu email
app.config['MAIL_PASSWORD'] = os.getenv('MAIL_PASSWORD')  # Tu contraseña o app password
app.config['MAIL_DEFAULT_SENDER'] = os.getenv('MAIL_DEFAULT_SENDER', 'noreply@tuapp.com')

mail = Mail(app)

# Configuración del Serializer para tokens
s = URLSafeTimedSerializer(app.config['JWT_SECRET_KEY'])
Explicación:

Configuración de Flask-Mail:

MAIL_SERVER: Servidor SMTP. Por defecto, Gmail.
MAIL_PORT: Puerto SMTP. 587 para TLS.
MAIL_USE_TLS: Uso de TLS.
MAIL_USERNAME y MAIL_PASSWORD: Credenciales de email para enviar correos.
MAIL_DEFAULT_SENDER: Dirección de correo predeterminada desde la cual se enviarán los emails.
Serializer:

URLSafeTimedSerializer de itsdangerous se utiliza para generar tokens seguros que expiran después de un tiempo determinado.
Nota: Si utilizas Gmail, es recomendable crear una App Password para permitir que tu aplicación envíe emails.

3. Actualizar el Modelo de Usuario para Incluir Confirmación de Email
Añade campos adicionales al modelo User para gestionar la confirmación de email.

python
Copiar
class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80), unique=True, nullable=False)
    email = db.Column(db.String(120), unique=True, nullable=False)  # Nuevo campo de email
    password = db.Column(db.String(128), nullable=False)
    email_confirmed = db.Column(db.Boolean, default=False)  # Indica si el email está confirmado
    email_confirmed_on = db.Column(db.DateTime, nullable=True)  # Fecha de confirmación

    def __repr__(self):
        return f'<User {self.username}>'
Explicación:

Campos Nuevos:
email: Almacena la dirección de correo electrónico del usuario. Debe ser única y no nula.
email_confirmed: Indica si el usuario ha confirmado su email.
email_confirmed_on: Registra la fecha y hora en que se confirmó el email.
Actualizar la Base de Datos:

Después de modificar el modelo, crea y aplica una nueva migración:

bash
Copiar
flask db migrate -m "Añadir campos de email al modelo de Usuario"
flask db upgrade
4. Crear Funciones para Generar y Verificar Tokens de Confirmación
Añade funciones auxiliares para manejar la generación y verificación de tokens de confirmación de email.

python
Copiar
def generate_confirmation_token(email):
    return s.dumps(email, salt='email-confirm')

def confirm_token(token, expiration=3600):
    try:
        email = s.loads(token, salt='email-confirm', max_age=expiration)
    except SignatureExpired:
        return None  # Token expirado
    except BadSignature:
        return None  # Token inválido
    return email
Explicación:

generate_confirmation_token(email): Genera un token seguro basado en la dirección de email del usuario, con un salt para mayor seguridad.
confirm_token(token, expiration): Verifica la validez del token y asegura que no haya expirado. Retorna el email si es válido, de lo contrario None.
5. Implementar Endpoints de Confirmación de Email
5.1. Enviar Email de Confirmación en el Endpoint de Registro
Actualiza el endpoint /register para enviar un email de confirmación después de que un usuario se registre exitosamente.

python
Copiar
@app.route('/register', methods=['POST'])
def register():
    """
    Endpoint para registrar nuevos usuarios.
    """
    data = request.get_json()
    username = data.get('username', None)
    password = data.get('password', None)
    email = data.get('email', None)  # Obtener el email del request
    
    if not username or not password or not email:
        return jsonify({"msg": "Username, email and password required"}), 400
    
    # Verificar si el usuario ya existe
    if User.query.filter_by(username=username).first():
        return jsonify({"msg": "Username already exists"}), 409
    if User.query.filter_by(email=email).first():
        return jsonify({"msg": "Email already registered"}), 409
    
    # Hash de la contraseña
    hashed_password = bcrypt.generate_password_hash(password).decode('utf-8')
    
    # Crear un nuevo usuario
    new_user = User(username=username, email=email, password=hashed_password)
    db.session.add(new_user)
    db.session.commit()
    
    # Generar el token de confirmación
    token = generate_confirmation_token(email)
    
    # Enviar el email de confirmación
    confirm_url = url_for('confirm_email', token=token, _external=True)
    html = render_template('activate.html', confirm_url=confirm_url)
    subject = "Por favor confirma tu email"
    
    msg = Message(recipients=[email],
                  subject=subject,
                  html=html)
    mail.send(msg)
    
    return jsonify({"msg": "User registered successfully. Por favor, confirma tu email."}), 201
Explicación:

Obtener Email del Request: Ahora, el endpoint espera un campo email en el JSON del request.
Verificación de Unicidad: Asegura que tanto username como email sean únicos.
Envío del Email de Confirmación:
Genera un token de confirmación basado en el email del usuario.
Crea una URL de confirmación que incluye el token.
Renderiza una plantilla HTML (activate.html) con el enlace de confirmación.
Envía el email al usuario con el enlace de confirmación.
5.2. Crear el Endpoint para Confirmar el Email
Añade un nuevo endpoint /confirm/<token> que maneje la confirmación del email.

python
Copiar
@app.route('/confirm/<token>')
def confirm_email(token):
    try:
        email = confirm_token(token)
    except:
        return jsonify({"msg": "El enlace de confirmación es inválido o ha expirado."}), 400
    
    user = User.query.filter_by(email=email).first_or_404()
    
    if user.email_confirmed:
        return jsonify({"msg": "Cuenta ya confirmada. Por favor inicia sesión."}), 200
    else:
        user.email_confirmed = True
        user.email_confirmed_on = datetime.utcnow()
        db.session.add(user)
        db.session.commit()
        return jsonify({"msg": "Has confirmado tu cuenta. Gracias!"}), 200
Explicación:

Verificación del Token:

Usa la función confirm_token para verificar la validez del token.
Si el token es inválido o ha expirado, retorna un error.
Actualizar el Estado del Usuario:

Busca al usuario por el email obtenido del token.
Si el usuario ya ha confirmado su email, informa al usuario.
Si no, actualiza los campos email_confirmed y email_confirmed_on, y confirma la cuenta.
Nota: En un entorno real, podrías redirigir al usuario a una página de confirmación en lugar de devolver un JSON. Para simplificar, aquí devolvemos mensajes JSON.

6. Actualizar el Frontend para Manejar la Confirmación de Email
6.1. Crear una Plantilla HTML para el Email de Confirmación
Crea un nuevo archivo activate.html en el directorio templates para el contenido del email de confirmación.

html
Copiar
<!-- templates/activate.html -->

<!DOCTYPE html>
<html>
<head>
    <meta charset="UTF-8">
    <title>Confirma tu Email</title>
</head>
<body>
    <p>Hola,</p>
    <p>Gracias por registrarte. Por favor, haz clic en el siguiente enlace para confirmar tu dirección de email:</p>
    <p><a href="{{ confirm_url }}">{{ confirm_url }}</a></p>
    <p>Si no creaste esta cuenta, por favor ignora este correo.</p>
</body>
</html>
Explicación:

Contenido del Email:
Un saludo.
Instrucciones para confirmar el email.
Enlace de confirmación generado dinámicamente.
Mensaje de precaución en caso de recepción no autorizada.
6.2. Actualizar el Frontend para Incluir el Campo de Email en el Registro
Actualiza el archivo templates/index.html para incluir un campo de email en el formulario de registro.

html
Copiar
<!-- templates/index.html -->

<!DOCTYPE html>
<html>
<head>
    <title>Clasificador de Dígitos MNIST</title>
    <script src="https://code.jquery.com/jquery-3.6.0.min.js"></script>
</head>
<body>

    <h1>Clasificador de Dígitos MNIST</h1>
    
    <!-- Formulario de Registro -->
    <h2>Registrar Usuario</h2>
    <form id="registerForm">
        <label for="reg_username">Usuario:</label>
        <input type="text" id="reg_username" name="username" required>
        <br>
        <label for="reg_email">Email:</label>
        <input type="email" id="reg_email" name="email" required>
        <br>
        <label for="reg_password">Contraseña:</label>
        <input type="password" id="reg_password" name="password" required>
        <br>
        <button type="submit">Registrar</button>
    </form>
    
    <div id="registerResult"></div>
    
    <!-- Formulario de Login -->
    <h2>Iniciar Sesión</h2>
    <form id="loginForm">
        <label for="username">Usuario:</label>
        <input type="text" id="username" name="username" required>
        <br>
        <label for="password">Contraseña:</label>
        <input type="password" id="password" name="password" required>
        <br>
        <button type="submit">Iniciar Sesión</button>
    </form>
    
    <div id="loginResult"></div>
    
    <!-- Formulario de Predicción -->
    <h2>Enviar Imagen para Predicción</h2>
    <input type="file" id="fileInput" accept="image/*">
    <button onclick="uploadImage()">Enviar para Predicción</button>

    <h2>Resultado:</h2>
    <p id="result"></p>

    <script>
        let accessToken = '';

        // Manejar el formulario de registro
        $('#registerForm').submit(function(event) {
            event.preventDefault();
            const username = $('#reg_username').val();
            const email = $('#reg_email').val();
            const password = $('#reg_password').val();

            $.ajax({
                url: '/register',
                type: 'POST',
                contentType: 'application/json',
                data: JSON.stringify({ username: username, email: email, password: password }),
                success: function(response) {
                    $('#registerResult').text('Registro exitoso. Por favor, confirma tu email.');
                },
                error: function(xhr, status, error) {
                    $('#registerResult').text('Error en el registro: ' + xhr.responseJSON.msg);
                }
            });
        });

        // Manejar el formulario de login
        $('#loginForm').submit(function(event) {
            event.preventDefault();
            const username = $('#username').val();
            const password = $('#password').val();

            $.ajax({
                url: '/login',
                type: 'POST',
                contentType: 'application/json',
                data: JSON.stringify({ username: username, password: password }),
                success: function(response) {
                    accessToken = response.access_token;
                    $('#loginResult').text('Inicio de sesión exitoso. Token obtenido.');
                },
                error: function(xhr, status, error) {
                    $('#loginResult').text('Error en el inicio de sesión: ' + xhr.responseJSON.msg);
                }
            });
        });

        function uploadImage() {
            if (!accessToken) {
                alert("Por favor, inicia sesión primero.");
                return;
            }

            var fileInput = document.getElementById('fileInput');
            var file = fileInput.files[0];
            if (!file) {
                alert("Por favor, selecciona una imagen.");
                return;
            }

            var formData = new FormData();
            formData.append('file', file);

            $.ajax({
                url: '/predict',
                type: 'POST',
                headers: {
                    'Authorization': 'Bearer ' + accessToken
                },
                data: formData,
                contentType: false,
                processData: false,
                success: function(response) {
                    if(response.error){
                        $('#result').text('Error: ' + response.error);
                    } else {
                        let cacheStatus = response.cached ? ' (Desde Cache)' : '';
                        $('#result').text('Predicción: ' + response.prediccion + ' | Probabilidad: ' + (response.probabilidad * 100).toFixed(2) + '%' + cacheStatus);
                    }
                },
                error: function(xhr, status, error) {
                    $('#result').text('Error: ' + xhr.responseJSON.error);
                }
            });
        }
    </script>

</body>
</html>
Explicación:

Campo de Email en el Registro:
Añadiste un campo de entrada para el email en el formulario de registro.
Mensajes de Confirmación:
Al registrarse exitosamente, el usuario recibe un mensaje indicando que debe confirmar su email.
Interacción con los Endpoints:
El frontend ahora maneja el registro con el campo de email y muestra mensajes apropiados según la respuesta del servidor.
7. Actualizar las Pruebas Automatizadas para la Confirmación de Email
Es crucial asegurarse de que el proceso de registro y confirmación de email funcione correctamente mediante pruebas automatizadas.

7.1. Actualizar test_api.py para Incluir Confirmación de Email
Añade nuevas pruebas para verificar que los usuarios deben confirmar su email antes de acceder a la API.

python
Copiar
# tests/test_api.py

import pytest
from api import app, db, User, generate_confirmation_token, confirm_token
import os
from io import BytesIO
from PIL import Image
import numpy as np
import json
from flask import url_for
from unittest.mock import patch

@pytest.fixture
def client():
    app.config['TESTING'] = True
    app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///:memory:'  # Usar una base de datos en memoria para pruebas
    app.config['MAIL_SUPPRESS_SEND'] = True  # Evita el envío de emails durante las pruebas
    with app.test_client() as client:
        with app.app_context():
            db.create_all()
            yield client
            db.session.remove()
            db.drop_all()

def generate_image(digit=5):
    # Genera una imagen en blanco con un dígito negro en el centro
    img = Image.new('L', (28, 28), color=255)
    # Aquí podrías dibujar el dígito, pero para simplicidad, dejaremos la imagen en blanco
    # En un caso real, usarías una imagen real de MNIST
    return img

def register_user(client, username, email, password):
    response = client.post('/register', json={
        'username': username,
        'email': email,
        'password': password
    })
    return response

def login(client, username, password):
    response = client.post('/login', json={
        'username': username,
        'password': password
    })
    data = response.get_json()
    return data.get('access_token')

@patch('api.mail.send')
def test_register_and_confirm_email(mock_send, client):
    # Registrar un nuevo usuario
    response = register_user(client, 'testuser', 'test@example.com', 'testpassword')
    assert response.status_code == 201
    json_data = response.get_json()
    assert json_data['msg'] == 'User registered successfully. Por favor, confirma tu email.'
    
    # Verificar que se envió un email de confirmación
    assert mock_send.called
    msg = mock_send.call_args[0][0]
    assert msg.recipients == ['test@example.com']
    assert 'Por favor confirma tu email' in msg.subject
    assert 'confirm_url' in msg.html

    # Obtener el token de confirmación
    user = User.query.filter_by(email='test@example.com').first()
    assert user is not None
    token = generate_confirmation_token(user.email)
    
    # Confirmar el email
    confirm_response = client.get(f'/confirm/{token}')
    assert confirm_response.status_code == 200
    confirm_json = confirm_response.get_json()
    assert confirm_json['msg'] == 'Has confirmado tu cuenta. Gracias!'
    
    # Verificar que el usuario está confirmado
    user = User.query.filter_by(email='test@example.com').first()
    assert user.email_confirmed == True
    assert user.email_confirmed_on is not None

def test_login_before_confirmation(client):
    # Registrar un nuevo usuario
    register_user(client, 'testuser', 'test@example.com', 'testpassword')
    
    # Intentar iniciar sesión antes de confirmar el email
    response = client.post('/login', json={
        'username': 'testuser',
        'password': 'testpassword'
    })
    # Según la implementación actual, el login aún está permitido, pero podrías modificarlo para requerir confirmación
    # Por ahora, verificamos que el token se genere
    assert response.status_code == 200
    json_data = response.get_json()
    assert 'access_token' in json_data

def test_confirm_email_invalid_token(client):
    # Intentar confirmar con un token inválido
    response = client.get('/confirm/invalidtoken')
    assert response.status_code == 400
    json_data = response.get_json()
    assert json_data['msg'] == 'El enlace de confirmación es inválido o ha expirado.'

def test_confirm_email_expired_token(client):
    # Generar un token con un tiempo de expiración corto
    with patch('api.confirm_token', return_value=None):
        response = client.get('/confirm/sometoken')
        assert response.status_code == 400
        json_data = response.get_json()
        assert json_data['msg'] == 'El enlace de confirmación es inválido o ha expirado.'
Explicación:

Mock de Envío de Emails:
Utiliza unittest.mock.patch para simular el envío de emails durante las pruebas, evitando el envío real.
Pruebas Incluidas:
test_register_and_confirm_email:
Registra un nuevo usuario.
Verifica que se envíe un email de confirmación.
Extrae y utiliza el token de confirmación para confirmar el email.
Verifica que el estado de email_confirmed del usuario se actualice correctamente.
test_login_before_confirmation:
Registra un usuario pero no confirma el email.
Intenta iniciar sesión y verifica que el login es posible.
Nota: Dependiendo de tus requisitos, podrías modificar el endpoint de login para requerir confirmación de email antes de permitir el acceso.
test_confirm_email_invalid_token:
Intenta confirmar un email con un token inválido y verifica que se maneja correctamente.
test_confirm_email_expired_token:
Simula la confirmación con un token expirado y verifica que se maneja correctamente.
Recomendación: Considera modificar el endpoint de login para restringir el acceso solo a usuarios que hayan confirmado su email.

8. Conclusiones y Recomendaciones
8.1. Implementación de Confirmación de Email
Beneficios:

Validación de Usuarios Reales: Asegura que los usuarios registrados poseen una dirección de email válida.
Prevención de Abusos: Reduce el riesgo de registros fraudulentos o automatizados.
Mejora de la Seguridad: Añade una capa adicional de verificación en el proceso de registro.
Recomendaciones:

Implementar Enlaces de Confirmación Seguros: Asegúrate de que los tokens de confirmación sean únicos y expiren después de un tiempo razonable.
Gestión de Errores: Proporciona mensajes claros y útiles a los usuarios en caso de fallos en la confirmación.
Ampliar Funcionalidades: Considera añadir la opción de reenviar el email de confirmación en caso de que el usuario no lo haya recibido.
8.2. Mejorar el Proceso de Login
Restricción de Acceso: Modifica el endpoint de login para que solo permita el acceso a usuarios que hayan confirmado su email.
python
Copiar
@app.route('/login', methods=['POST'])
def login():
    """
    Endpoint para que los usuarios inicien sesión y obtengan un token JWT.
    """
    data = request.get_json()
    username = data.get('username', None)
    password = data.get('password', None)
    
    if not username or not password:
        return jsonify({"msg": "Username and password required"}), 400
    
    # Buscar al usuario en la base de datos
    user = User.query.filter_by(username=username).first()
    
    if not user or not bcrypt.check_password_hash(user.password, password):
        return jsonify({"msg": "Bad username or password"}), 401
    
    if not user.email_confirmed:
        return jsonify({"msg": "Por favor, confirma tu email antes de iniciar sesión."}), 403
    
    # Crear el token de acceso
    access_token = create_access_token(identity=username)
    return jsonify(access_token=access_token), 200
Explicación:

Verificación de Confirmación de Email: Antes de generar el token JWT, verifica que el usuario haya confirmado su email. Si no, retorna un error indicando que debe confirmar su email.
8.3. Mejorar la Experiencia del Usuario en el Frontend
Mensajes Claros: Asegúrate de que el frontend informe al usuario sobre el estado de su registro y confirmación.
Reenvío de Emails de Confirmación: Proporciona una opción para que los usuarios puedan solicitar el reenvío del email de confirmación si no lo recibieron.
Redirección Después de Confirmar Email: Considera redirigir al usuario a una página de éxito o login después de confirmar su email.
8.4. Seguridad Adicional
Protección contra CSRF: Implementa protección contra ataques de Cross-Site Request Forgery en los formularios.
Validación de Entradas: Asegura que todas las entradas del usuario sean validadas y sanitizadas adecuadamente.
Uso de HTTPS: Asegúrate de que la aplicación se sirva sobre HTTPS para proteger la transmisión de datos sensibles.
9. Recursos Adicionales
Flask-Mail Documentation: Flask-Mail Docs
itsdangerous Documentation: itsdangerous Docs
Flask-JWT-Extended Documentation: Flask-JWT-Extended Docs
Flask-SQLAlchemy Documentation: Flask-SQLAlchemy Docs
Flask-Migrate Documentation: Flask-Migrate Docs
Flask-Bcrypt Documentation: Flask-Bcrypt Docs
PyTest Documentation: PyTest Docs
SQLAlchemy Documentation: SQLAlchemy Docs
Flask Testing Documentation: Flask Testing Docs
GitHub Actions Documentation: GitHub Actions Docs
Heroku Documentation: Heroku Docs
Conclusión
En el Día 25, has fortalecido la seguridad y gestión de usuarios de tu API mediante la implementación de la confirmación de email. Esto asegura que solo usuarios con direcciones de correo electrónico válidas puedan acceder y utilizar la API, protegiendo la aplicación contra registros fraudulentos y mejorando la integridad de la base de datos de usuarios.

Pasos Clave Realizados:

Configuración de Flask-Mail:

Integraste Flask-Mail para enviar correos electrónicos de confirmación a los usuarios registrados.
Actualización del Modelo de Usuario:

Añadiste campos email, email_confirmed y email_confirmed_on para gestionar la confirmación de email.
Generación y Verificación de Tokens:

Implementaste funciones para generar tokens seguros y verificar su validez.
Implementación de Endpoints de Confirmación:

Creaste el endpoint /confirm/<token> para manejar la confirmación de email.
Actualización del Frontend:

Añadiste campos de email en el formulario de registro y manejaste mensajes de confirmación en el frontend.
Actualización de Pruebas Automatizadas:

Añadiste pruebas para asegurar que el proceso de registro y confirmación de email funciona correctamente.
Recomendaciones para Continuar:

Implementar Funcionalidades de Reenvío de Confirmación:

Permite a los usuarios reenviar el email de confirmación si no lo recibieron.
Añadir Confirmación de Email Obligatoria para el Login:

Asegura que solo los usuarios que hayan confirmado su email puedan iniciar sesión.
Implementar Funcionalidades Adicionales de Seguridad:

Considera agregar protección contra CSRF, validación avanzada de entradas y uso de HTTPS.
Optimizar la Gestión de Usuarios:

Añade roles y permisos para diferentes tipos de usuarios (e.g., admin, user).
Mejorar la Experiencia del Usuario:

Implementa redirecciones y mensajes claros en el frontend para guiar al usuario durante el proceso de registro y confirmación.